<a href="https://colab.research.google.com/github/smartgrids-aau/GreenCodesPython/blob/main/Exemple_6_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# Exercise 6.3

import numpy as np
import pandas as pd
from numpy import pi, tan, abs, sqrt, real, imag
import warnings

warnings.filterwarnings("ignore")

# File and sheet
fileName = 'PV Modeling Book Data Source.xls'
sheetName = 'Source 7'

# Data reading
G = pd.read_excel(fileName, sheet_name=7, usecols='I', skiprows=9, nrows=4380).values.flatten()
Tc = pd.read_excel(fileName, sheet_name=7, usecols='J', skiprows=9, nrows=4380).values.flatten()
Vm = pd.read_excel(fileName, sheet_name=7, usecols='L', skiprows=9, nrows=4380).values.flatten()
Im = pd.read_excel(fileName, sheet_name=7, usecols='M', skiprows=9, nrows=4380).values.flatten()

CNs = np.zeros(80000)
CNp = np.zeros(80000)
CCn = np.zeros(80000)
CLLP = np.ones(80000) * -0.5
CQexcess = np.ones(80000) * -0.5
CQdeficit = np.ones(80000) * -0.5
CQ = np.ones(80000) * -0.5

q = 0

for Cn in range(1, 3):  # Reduced range for demonstration (use range(1, 121) for full run)
    for N in range(6, 8):  # Reduced range for demonstration (use range(6, 33) for full run)
        for Ns in range(1, N + 1):
            if N % Ns == 0:
                Np = N // Ns
                h1 = 20
                h2 = 0.1444

                Va = Ns * Vm
                Ia = Np * Im
                Vpv = Va
                Ipv = Ia
                Pao = Va * Ia
                Am = 0.9291
                A = Ns * Np * Am
                Pai = A * G
                with np.errstate(divide='ignore', invalid='ignore'):
                    effa = np.nan_to_num(Pao / Pai)

                Va = 0.95 * Va
                Ia = 0.90 * Ia
                Ra = 0.8
                Km = 0.175
                Ebb = Va - Ra * Ia
                Eb = (Ebb >= 0) * Ebb
                Ia = (Ebb >= 0) * Ia
                Va = (Ebb >= 0) * Va
                Tm = Km * Ia
                Tmm = (Tm == 0) * 1
                Tm1 = Tmm + Tm

                Rou = 1000
                d1 = 33.5 * 0.001
                d2 = 160 * 0.001
                beta1 = 38 * 2 * pi / 360
                beta2 = 33 * 2 * pi / 360
                b1 = 5.4 * 0.001
                b2 = 2.2 * 0.001
                Kp = Rou * 2 * pi * b1 * (d1 / 2) ** 2 * tan(beta1) * ((d2 / 2) ** 2 - ((b1 * (d1 / 2) ** 2 * tan(beta1)) / (b2 * tan(beta2))))

                Pdev = Eb * Ia
                with np.errstate(divide='ignore', invalid='ignore'):
                    Omega = np.nan_to_num(sqrt((Km * Ia) / Kp))
                Pmo = Pdev
                Pmi = Pao * 0.9
                PMI1 = (Pmi == 0) * 1
                PMI2 = Pmi + PMI1
                with np.errstate(divide='ignore', invalid='ignore'):
                    effm = np.nan_to_num(Pmo / PMI2)

                Tp = Tm
                Eh = Tp * Omega
                Ppo = Eh
                Ppo = (Ebb >= 0) * Ppo
                Ppi = Pmo
                PPI1 = (Ppi == 0) * 1
                PPI2 = Ppi + PPI1
                with np.errstate(divide='ignore', invalid='ignore'):
                    effpp = np.nan_to_num(Ppo / PPI2)
                    effp = (effpp <= 0.95) * effpp

                QQ = np.zeros(len(Eh))
                for ii in range(len(Eh)):
                    coeffs = [h2 * 2.725, 0, h1 * 2.725, -Eh[ii]]
                    r = np.roots(coeffs)
                    QQQ = 0
                    for root in r:
                        if imag(root) == 0 and real(root) > 0:
                            QQQ = real(root)
                            break
                    QQ[ii] = QQQ

                Q1 = (QQ == 0) * 1
                Q2 = QQ + Q1
                with np.errstate(divide='ignore', invalid='ignore'):
                    H = np.nan_to_num(Eh / (2.725 * Q2))

                effsub = effm * effp
                effoverall = effa * effm * effp
                QQ = (Ebb >= 0) * QQ
                QQ = (effpp <= 0.95) * QQ
                Q = np.concatenate(([0], QQ))
                d = 2.5
                Cr = np.zeros(len(Q))
                Qexcess_pv = np.zeros(len(Q))
                Qexcess_s = np.zeros(len(Q))
                SOC = np.zeros(len(Q))
                Qdef_pv = np.zeros(len(Q))
                Qdeficit_s = np.zeros(len(Q))
                X = Q[1:] - d

                Qdef_pv[1:] = (X < 0) * abs(X)
                Qexcess_pv[1:] = (X >= 0) * abs(X)

                C = len(Q) - 1
                for i in range(C):
                    Cr[i+1] = ((Cr[i] + X[i]) >= 0) * abs(Cr[i] + X[i])
                    SOC[i+1] = Cr[i+1] / Cn
                    if SOC[i+1] >= 1:
                        SOC[i+1] = 1
                        Qexcess_s[i+1] = Cr[i+1] - Cn
                        Cr[i+1] = Cn
                    else:
                        Qexcess_s[i+1] = 0
                    Qdeficit_s[i+1] = ((Cr[i] + X[i]) < 0) * abs(Cr[i] + X[i])

                Q = Q[1:]
                sumQ = np.sum(Q)
                Qexcess = np.sum(Qexcess_s[1:])
                Qdeficit = np.sum(Qdeficit_s[1:])
                D = np.full(len(Q), d)
                with np.errstate(divide='ignore', invalid='ignore'):
                    LLP = np.sum(Qdeficit_s[1:]) / np.sum(D)

                if LLP <= 0.01:
                    CNs[q] = Ns
                    CNp[q] = Np
                    CCn[q] = Cn
                    CLLP[q] = LLP
                    CQexcess[q] = Qexcess
                    CQdeficit[q] = Qdeficit
                    CQ[q] = sumQ
                    q += 1

CNs_result = CNs[:q]
CNp_result = CNp[:q]
CCn_result = CCn[:q]
CLLP_result = CLLP[:q]
CQ_result = CQ[:q]

CNs_result, CNp_result, CCn_result, CLLP_result, CQ_result


ParserError: Defining usecols with out-of-bounds indices is not allowed. [8] are out-of-bounds. (sheet: 7)